## Set Up

In [ ]:
# Clone the repository from github if not already setup
import os

dir_path = '/content/rPPG-Toolbox'
if os.path.isdir(dir_path):
  print(f"The directory '{dir_path}' exists.")
else:
  !git clone https://github.com/ubicomplab/rPPG-Toolbox.git
%cd '{dir_path}'
!ls '{dir_path}'

Cloning into 'rPPG-Toolbox'...
remote: Enumerating objects: 3508, done.
remote: Counting objects: 100% (1371/1371), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 3508 (delta 1302), reused 1255 (delta 1255), pack-reused 2137 (from 2)
Receiving objects: 100% (3508/3508), 335.12 MiB | 28.61 MiB/s, done.
Resolving deltas: 100% (2424/2424), done.
Updating files: 100% (288/288), done.
/content/rPPG-Toolbox
config.py  evaluation		LICENSE        neural_methods	 setup.sh	       wip
configs    figures		main.py        README.md	 tools
dataset    final_model_release	model_outputs  requirements.txt  unsupervised_methods


In [ ]:
# Explicitly install PyQt5 to use interactive plotting and avoid non-interactive backends
# See this relevant issue for more details: https://github.com/astral-sh/uv/issues/6893
# This requires Python 3.9+
!which python
!python --version
!pip install  --no-build-isolation PyQt5

/usr/local/bin/python
Python 3.10.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.8/270.8 kB 26.1 MB/s eta 0:00:00


In [ ]:
### Setup using uv
### This gives an error when building the mamba package.
### Hence we need to directly install causal-conv1d and mamba-ssm.
### See README file under tools/mamba.
#!bash setup.sh uv
%%bash
rm -rf .venv
uv venv --python 3.8
source .venv/bin/activate
uv pip install --no-build-isolation setuptools wheel
uv pip install --no-build-isolation torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
uv pip install --no-build-isolation -r requirements.txt
uv pip install --no-build-isolation causal-conv1d
uv pip install --no-build-isolation mamba-ssm


Using CPython 3.8.20
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 2 packages in 161ms
Prepared 2 packages in 128ms
Installed 2 packages in 64ms
 + setuptools==75.3.0
 + wheel==0.45.1
Resolved 19 packages in 1.81s
Prepared 19 packages in 43.08s
Installed 19 packages in 412ms
 + certifi==2022.12.7
 + charset-normalizer==2.1.1
 + filelock==3.13.1
 + fsspec==2024.2.0
 + idna==3.4
 + jinja2==3.1.3
 + markupsafe==2.1.5
 + mpmath==1.3.0
 + networkx==3.2.1
 + numpy==1.24.1
 + pillow==10.2.0
 + requests==2.28.1
 + sympy==1.13.1
 + torch==2.1.2+cu121
 + torchaudio==2.1.2+cu121
 + torchvision==0.16.2+cu121
 + triton==2.1.0
 + typing-extensions==4.9.0
 + urllib3==1.26.13
Resolved 126 packages in 2.40s
Prepared 111 packages in 1m 07s
Uninstalled 5 packages in 210ms
Installed 111 packages in 926ms
 + absl-py==2.1.0
 + asttokens==3.0.0
 + astunparse==1.6.3
 + backcall==0.2.0
 + beautifulsoup4==4.12.3
 + cachetools==5.5.0
 + causal-conv1d==1.0.0
 + comm==0.2

In [ ]:
### List packages in the virtual environment
!source .venv/bin/activate
!uv pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
asttokens                    3.0.0
astunparse                   1.6.3
backcall                     0.2.0
beautifulsoup4               4.12.3
cachetools                   5.5.0
causal-conv1d                1.0.0
certifi                      2022.12.7
charset-normalizer           2.1.1
comm                         0.2.2
cycler                       0.12.1
debugpy                      1.8.11
decorator                    5.1.1
einops                       0.8.0
executing                    2.1.0
filelock                     3.13.1
flatbuffers                  24.12.23
fsspec                       2024.10.0
gast                         0.4.0
gdown                        5.2.0
google-auth                  2.37.0
google-auth-oauthlib         1.0.0
google-pasta                 0.2.0
grpcio                       1.69.0
h5py                         2.10.0
huggingface-hub              

# Inference on pretrained models

###  Steps:

#### 1. Run PhysNet on UBFC to test that it works
#### 2. Conduct “studies”, ie try compressing the dataset and measuring UBFC performance as the compression ratio increases, graph that
#### 3. Try experimenting with training compressed data
#### 4. Read these papers: https://arxiv.org/pdf/1905.02419,  https://arxiv.org/abs/2111.12082

In [ ]:
### UBFC-rPPG dataset in drive folder
### (TODO) Figure out how to import data set directly into colab
from google.colab import drive

drive.mount('/content/drive')
!ls -l /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_2/subject9

Mounted at /content/drive
total 1814623
-r-------- 1 root root      96774 Mar  7  2018 ground_truth.txt
-r-------- 1 root root 1858075976 Nov  2  2017 vid.avi


In [ ]:
### Compress the UBFC dataset and store the compressed files under a separate
### dir structure. Only needs to be run once.
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import shutil

!pip install -q mediapy
import mediapy as media

base_path = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_2'
compressed_path = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3'

COMPRESSION_BIT_RATE = 10_000_000

dirnames = os.listdir(base_path)
if not os.path.isdir(compressed_path):
  os.mkdir(compressed_path)
for dirname in dirnames:
  if not os.path.isdir(os.path.join(compressed_path, dirname)):
    os.mkdir(os.path.join(compressed_path, dirname))
  for filename in os.listdir(os.path.join(base_path, dirname)):
    if filename.endswith('.avi'):
      video = media.read_video(os.path.join(base_path, dirname, filename))
      compressed_data = media.compress_video(video, bps=COMPRESSION_BIT_RATE)
      print("Writing " + os.path.join(compressed_path, dirname, 'vid.mp4'))
      with open(os.path.join(compressed_path, dirname, 'vid.mp4'), 'wb') as f:
        f.write(compressed_data)
    else:
      shutil.copy(os.path.join(base_path, dirname, filename), os.path.join(compressed_path, dirname, filename))
print("All files compressed")

Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject1/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject10/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject11/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject12/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject13/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject14/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject15/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject16/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_3/subject17/vid.mp4


In [ ]:
### Inference on UBFC-RPPG dataset using a model trained on PURE
###
### Update the YAML file under configs dir before running this
### TOOLBOX_MODE: "only_test"
### TEST.DATA.DATA_PATH = "/content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_2"
### TEST.DATA.CACHED_PATH: "/content/drive/MyDrive/research_project/PreprocessedData"
### TEST.OUTPUT_SAVE_DIR: "/content/drive/MyDrive/research_project/OutputData"
### LOG.PATH:  "/content/drive/MyDrive/research_project/runs/exp"
### TEST.DATA.DO_PREPROCESS: True  (only the first time running this)
###
### *****To use compressed data, need to update the UBFC data loader to read from the .mp4 file
### Also, change DATASET_2 to DATASET_3

!source .venv/bin/activate && python main.py --config_file ./configs/infer_configs/PURE_UBFC-rPPG_PHYSNET_BASIC.yaml

=> Merging a config file from ./configs/infer_configs/PURE_UBFC-rPPG_PHYSNET_BASIC.yaml
Configuration:
BASE: ['']
DEVICE: cuda:0
INFERENCE:
  BATCH_SIZE: 4
  EVALUATION_METHOD: FFT
  EVALUATION_WINDOW:
    USE_SMALLER_WINDOW: False
    WINDOW_SIZE: 10
  MODEL_PATH: ./final_model_release/PURE_PhysNet_DiffNormalized.pth
LOG:
  PATH: /content/drive/MyDrive/research_project/runs/exp
MODEL:
  BIGSMALL:
    FRAME_DEPTH: 3
  DROP_RATE: 0.2
  EFFICIENTPHYS:
    FRAME_DEPTH: 10
  MODEL_DIR: /content/drive/MyDrive/research_project/runs/exp/_SizeW128_SizeH128_ClipLength180_DataType_DataAugNone_LabelType_Crop_faceTrue_BackendHC_Large_boxTrue_Large_size1.5_Dyamic_DetFalse_det_len30_Median_face_boxFalse/PreTrainedModels
  NAME: Physnet
  PHYSFORMER:
    DIM: 96
    FF_DIM: 144
    NUM_HEADS: 4
    NUM_LAYERS: 12
    PATCH_SIZE: 4
    THETA: 0.7
  PHYSNET:
    FRAME_NUM: 128
  RESUME: 
  TSCAN:
    FRAME_DEPTH: 10
  iBVPNet:
    CHANNELS: 3
    FRAME_NUM: 160
NUM_OF_GPU_TRAIN: 1
TEST:
  DATA:
    BEG

# New Section